# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2022 &ndash; Week 3 &ndash; ETH Zurich</center>

## Introduction
This week we will cover mostly theoretical aspects of Hadoop and HDFS and we will discuss advantages and limitations of different storage models.

#### What is Hadoop?
Hadoop provides a **distributed file system** and a
**framework for the analysis and transformation** of very **large**
data sets using the MapReduce paradigm.

Several components are part of this framework. In this course you will study HDFS, MapReduce and HBase while this exercise focuses on HDFS and storage models.


| *Component*                |*Description*  |*First developer*  |
|----------------------------------------------|---|---|
| **HDFS**                  |Distributed file system  |Yahoo!  |
| **MapReduce**   |Distributed computation framework   |Yahoo!  |
| **HBase**           | Column-oriented table service  |Powerset (Microsoft)  |
| Pig  | Dataflow language and parallel execution framework  |Yahoo!   |
| Hive            |Data warehouse infrastructure   |Facebook  |
| ZooKeeper    |Distributed coordination service   |Yahoo!  |
| Chukwa  |System for collecting management data   |Yahoo!  |
| Avro                |Data serialization system   |Yahoo! + Cloudera  |

## 1. The Hadoop Distributed File System
### 1.1 &ndash; State which of the following statements are true:

1. The HDFS namespace is a hierarchy of files and directories.
True

1. In HDFS, each block of the file is either 64 or 128 megabytes depending on the version and distribution of Hadoop in use, and this *cannot* be changed.
False.

1. A client wanting to write a file into HDFS, first contacts the NameNode, then sends the data to it. The NameNode will write the data into multiple DataNodes in a pipelined fashion.
False.

1. A DataNode may execute multiple application tasks for different clients concurrently.
True.

1. The cluster can have thousands of DataNodes and tens of thousands of HDFS clients per cluster.
True.

1. HDFS NameNodes keep the namespace in RAM.
True.

1. The locations of block replicas are part of the persistent checkpoint that the NameNode stores in its native file system.
False.

1. If the block size is set to 64 megabytes, storing a file of 80 megabytes will actually require 128 megabytes of physical memory (2 blocks of 64 megabytes each).
False.

### 1.2 &ndash; A typical filesystem block size is 4096 bytes. How large is a block in HDFS? List at least two advantages of such choice.
A: A block size in HDFS is either 64MB or 128MB. One advantage to this block size is that files can be broken up into relatively small sized chunks resulting in multiple blocks being replicated across the nodes, offering more reliability and space usage for each file. Another advantage has to do with transfering blocks over the network. With a smaller block size, the required blocks for a file would be much higher, incurring a rather high latency for fetching the required file blocks for operations. On the other hand, having a bigger block size would make it innefficient to send the blocks across network, as they might not be able to serve gigabytes of data for each transfer. Therefore, this sizing allows for better throughput at a good latency cost and also offers reliabilirt as lost packets are easier to be resent.

### 1.3 &ndash; How does the hardware cost grow as function of the amount of data we need to store in a Distributed File System such as HDFS? Why?
A: Distributed file systems such as HDFS are built on relatively cheap hardware. In the case of HDFS, the cost of storing data mostly depends on the price of storage systems. Using HDDs would come with the benefit of low costs at an icnreased latency for accessing blocks. However, this is a cost a system such as HDFS will allow due to the added benefits of it, such as consistency and availability. Another thing to point out is that for each stored file, we are going to copy the blocks to at least 2 other locations, meaning that for a file of size X, we actually use at least 3 * x size, without the added storage requirements for metadata and hierarchy incurred on the namenode.

Therefore, one can conclude hardware costs grow linearly as a function of amount of data.

### 1.4 &ndash; Single Point of Failure

1. Which component is the main single point of failure in Hadoop?
A: The NameNode is the single point of failure in Hadoop, as it is the only node responsible for storing the file hierarchy, the mappings from files to blocks and the block locations, thought the last piece of information could always be recomputed.

1. What is the Secondary NameNode?
To solve the issue of single point of failure, a secondary NameNode can be introduced. The NameNode then will write an fsImage containing the file hierarchy and file-to-block mappings into durable storage, while also adding entries to an EditLog for keeping track of what operations have been performed since the last fsImage was built. The secondary NameNode looks at these two files, and a given time period will create a new fsImage using the EditLog information, resulting in an updated fsImage called a checkpoint. These checkpoints help reduce the size of EditLogs and keep the fsImage up to date to the latest changes.

### 1.5 &ndash; Scalability, Durability and Performance on HDFS
Explain how HDFS accomplishes the following requirements:

1. Scalability
Scalability is achieved by HDFS using a centralized approach, with enforced independency among nodes that store data. Moreover, it achieves scalability by clearly defining the responsibilities of each node. Therefore, adding new data storage is a matter of creating a new process on a new machine that runs as a DataNode. The system will register this new node as the DataNode will ping the NameNode with updates abouts its health and readiness. On the other hand, turning nodes off has the effect that NameNodes will register the disappearance of them, and will use their replication mechanism such that enough replicas of the files that resided on the turned off nodes are available.

1. Durability
Due to each file's block being stored in multiple DataNodes, across different racks, HDFS provides durability in case of hardware failure. The idea behind this mechanism is that each block has to be available in at least N locations to enforce its persistency. Moreover, these locations have a specific way in which they are chosen such that hardware failure has the minimum effect on losing the data at any given time. A replica will be stored across racks to minimize rack failure while also improving the speed at which data is delivered.

1. High sequential read/write performance
To achieve high performance, HDFS relies on the physical locations of the file blocks, the paralellism of reads/writes and block size. Firstly, the locations for storing file blocks are chosen in an efficient manner such that the distance between nodes is minimized. This results in less network delay incurred by sending the packets over long distances. Secondly, due to file blocks residing on different nodes in different racks, clients can be served from multiple locations, resulting in work being distributed over a higher number of nodes, miniziming the amount of work a single node has to do. Finally, the block size allows for fast delivery of packets over the network and small recovery time due to packet loses.

## 2. File I/O operations and replica management.


### 2.1 &ndash; Replication policy
Assume your HDFS cluster is made of 3 racks, each containing 3 DataNodes. Assume also the HDFS is configured to use a block size of 100 megabytes and that a client is connecting from outside the datacenter (therefore no DataNode is priviledged). 

1. The client uploads a file of 150 megabytes. Draw in the picture below a possible blocks configuration according to the default HDFS replica policy. How many replicas are there for each block? Where are these replicas stored?
A: The policy is based on the following mechanism:
    - A random node is selected for the first replica
    - The second replica is written on the closest rack, on a random node.
    - The third replica is written on the same rack as the second one, but on a different node.

1. Can you find a different policy that, using the same number of replicas, improves the expected availability of a block? Does your solution show any drawbacks?
A: One such policy is to store each replica in a sepparate rack. This will increase availability at the cost of increased time required for writing, as more inter-rach communication rounds have to be established for this process. The probability of a rack failure is much smaller than that of a single node, therefore the initial policy seems a better approach.
1. Referring to the picture below, assume a block is stored in Node 3, as well as in Node 4 and Node 5. If this block of data has to be processed by a task running on Node 6, which of the three replicas will be actually read by Node 6? 
A: Due to nodes 4 and 5 residing on the same rack as node 6, the read process will target either of these two. Assuming the distance to them is considered equal as they are in the same rack (incurring the same intra-rack cost), either of nodes 4 or 5 will be selected.

<img src="https://polybox.ethz.ch/index.php/s/lRzwDdtmytzyDRR/download" width="500">

### 2.2 &ndash; File read and write data flow.
To get an idea of how data flows between the client interacting with HDFS, consider a diagram below which shows main components of HDFS. 

<img src="https://polybox.ethz.ch/index.php/s/R7hg8x7YEyTFPvD/download" width="600">

1. Draw the main sequence of events when a client copies a file to HDFS.
2. Draw the main sequence of events when a client reads a file from HDFS.
3. Why do you think a client writes data directly to datanodes instead of sending it through the namenode?

### 2.3 &ndash; Network topology.
HDFS estimates the network bandwidth between two nodes by their distance. The distance from a node to its parent node is assumed to be one. A distance between two nodes can be calculated by summing up thier distances to their closest common ancestor. A shorter distance between two nodes means that the greater bandwidth they can utilize to transfer data. Consider a diagram of a possible hadoop cluster over two datacenters below. 

<img src="https://polybox.ethz.ch/index.php/s/Mk2kI7dkKZNrxul/download" width="700">

Calculate following distances using the distance rule explained above:
1. Node 0 and Node 1
2. Node 0 and Node 2
3. Node 1 and Node 4
4. Node 4 and Node 5
5. Node 2 and Node 3
6. Two processes of Node 1

1. 2
2. 4
3. 6
4. 4
5. 2
6. 0

## 3. Storage models
### 3.1 &ndash; List two differences between Object Storage and Block Storage.
1. Object Storage identifies objects using some form of key, such as bucket Id and user ID. On the other hand, block storage uses a file hierarchy for storing and accessing files.
2. Files stored in Object Storage are stored as one big file, whereas Block Storage is based on blocks, where each block file get split into one or more blocks that can then be shared across different machines.

### 3.2 &ndash; Compare Object Storage and Block Storage. For each of the following use cases, say which technology better fits the requirements.

1. Store Netflix movie files in such a way they are accessible from many client applications at the same time [ *Object storage | Block Storage* ]

1. Store experimental and simulation data from CERN [ *Object storage | Block Storage* ]

1. Store the auto-backups of iPhone/Android devices [ *Object storage | Block Storage* ]

1. Object Storage. Movies are not big files that would benefit from the block storage approach of Block Storage. On the other hand, for many client applications cocnurrent access, sending a single file over has a bigger benefit than trying to find the blocks every time, across different locations of the blocks and then sending them to each client.
2. Block Storage. This data is known to be humongous, therefore Object Storage will not be able to store a single huge file that is over 5TB. Moreover, the data from CERN is mostly going to require appending new information to existing files, which is not possible with Object Storage, but is what Block Storage is based upon.
3. Object Storage. Backups should not be that big, and would not require to be read that often. Storing the data in Object Storage will reduce the cost for the storage of the file - as Block Storage would require multiple replicas for each of the block files. Finally, these backups are write-once, and therefore would not benefit from the append functionality of the Block Storage systems.

## 4. Working Docker-Hadoop

Build and run the Hadoop docker image by `docker-compose up -d` in the `exercise03` directory. If completed successfully, you should be able to browse [`http://localhost:9870/`](http://localhost:9870/) and visualize the web interface of the daemon which should look similar to the following image. In the `Datanodes` tab you should see a single operating datanode.

<img src="https://polybox.ethz.ch/index.php/s/LpWcGWZeU5mipBK/download" width="800">


### Connecting to containers  

Each Hadoop cluster is set up in one of the three supported modes:

- Local (Standalone) Mode
- Pseudo-Distributed Mode
- Fully-Distributed Mode

By default Hadoop runs in Local Mode but we will run it in the *Pseudo-Distributed Mode*. This will allow you to run Hadoop on a single-node (your computer) simulating a distributed file system, with datanode and namenode running in separate containers. For this excercise you will only need to connect to `namenode` and `datanode` containers. To connect to namenode container can use the Docker dashboard interface by navigating to `docker-hadoop` app, and selecting `CLI` option from the `namenode` container (see image below).

<img src="https://polybox.ethz.ch/index.php/s/Hdlyhagx3JWbLBy/download" width="700">

Alternatively, you can run `docker exec -it namenode /bin/bash` in a terminal. To connect to a datanode, you can similarly find it in the dashboard or run `docker exec -it namenode /bin/bash` in the terminal. Both approaches will give you shell access on the corresponding container. 

### 4.1 &ndash; Upload a file into HDFS

Connect to the namenode by `docker exec -it namenode /bin/bash`.

Pick an image file in your computer (or you can also download a random one) and try to upload it to HDFS. You may need to create an empty directory before uploading. (Check [here](https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html) for help.)

1. Which command do you use to upload from the local file system to HDFS?

1. Which information can you find if you use `Utilities -> Browse the file system` in the daemon web interface?

### 4.2 &ndash; Local File System

1. ```bash
   docker cp docker-compose.yml namenode:docker-compose.yml 
   ```
Then, use HDFS commands to create a directory, copy the `docker-compose.yml` file from your local file system to HDFS. Use `cat` to check if the file is the same on the local and distributed systems. 

   *Hint:* you may use the following HDFS commands `-mkdir` for directory, `-copyFromLocal` for uploading the file, and `-cat` for printing them. You may have to first use `docker cp` to copy to file into the namenode container.

2. Try to locate the file on a datanode. To connect to a datanode by running:

   ```bash
   docker exec -it datanode /bin/bash
   ```

   This will give you shell access to the data node machine. cd into `/hadoop/dfs/data/current/` directory and follow the directories until there are only files. Can you check if the file contents are the same as the one you uploaded? Use `ls -l` to check the size of the file size on the local 

3. Now try to upload a file to HDFS that is ~150MB. On Unix-based system you can also generate such a file filled with zero using:

   ```bash
   dd if=/dev/zero of=zeros.dat bs=1M count=150
   ```

   How many blocks the file is split into?

### 4.3 Demystifying FsImage & Edits, & Checkpoint

When the NameNode starts up, or a checkpoint is triggered by a configurable threshold:

- It reads the FsImage and EditLog from disk.
- It applies all the transactions from the EditLog to the in-memory representation of the FsImage.
- It flushes out this new version into a new FsImage on disk.
- It truncates the old EditLog because its transactions have been applied to the persistent FsImage.

A checkpoint can be triggered:

> at a given time interval (dfs.namenode.checkpoint.period) expressed in seconds,
> or after a given number of filesystem transactions have accumulated (dfs.namenode.checkpoint.txns).

If both of these properties are set, the first threshold to be reached triggers a checkpoint.

1. Query the configuration file

   - `hdfs getconf -confKey dfs.namenode.checkpoint.period` - 3600s
   - `hdfs getconf -confKey dfs.namenode.checkpoint.txns` - 1000000
   - The fsimage & edit logs location `hdfs getconf -confKey dfs.namenode.name.dir`, I get something like `file:///hadoop/dfs/name`
   - Find the fsimage and edit logs in the `current` directory. They must be named like `fsimage_0000000000000000000` & `edits_inprogress_0000000000000000001` 
   - Output edits `hdfs oev -p xml -i /hadoop/dfs/name/current/edits_inprogress_0000000000000000001 -o edits.xml `
   - Output fsimage `hdfs oiv -p XML -i /hadoop/dfs/name/current/fsimage_0000000000000000000 -o fsimage.xml`

2. Can you make sense of the outputs?

### 4.4 Changing Block Size (optional)

As explained in the tutorials, to change HDFS configurations you edit `etc/hadoop/core-site.xml` and `etc/hadoop/hdfs-site.xml`. In the docker app, you can modify the variables in the `hadoop.env`. For example, in the following line,

```bash 
# hadoop.env 
CORE_CONF_fs_defaultFS=hdfs://namenode:9000
```

`CORE_CONF` corresponds to `core-site.xml`. The second part `fs_defaultFS=hdfs://namenode:9000` will be transformed into:

```xml
<property>
    <name>fs.defaultFS</name><
    value>hdfs://namenode:9000
    </value>
</property>
```

For more details [see here](https://github.com/big-data-europe/docker-hadoop).

Try changing the default block size of HDFS to see its affect on read & write performance. You can change the block size by modifying the follwoing line in `hadoop.env`: `HDFS_CONF_dfs_block_size=1048576` The value `1048576` determines the block size in bytes, which in this case is `2^20 bytes` or 1 megabytes.

> **_NOTE:_** that for these configuration changes to take effect you must restart the docker app!

1. Create a file with size ~150MB and uploade the file to HDFS. Check number of blocks via the Web interface. 

2. For each of the following block sizes 1048576, 134217728, measure the time to transfer from local to HDFS, from HDFS to local, and removing the file. 

You can run the following commands 
```bash
time hadoop fs -copyFromLocal zeros.dat /myfolder/zeros.dat
time hadoop fs -get /myfolder/zeros.dat zeros.dat
time hadoop fs -rm /myfolder/zeros.dat
```
Can you make sense of the results?

> **_NOTE:_** make sure to remove the files before uploading them, so that no caching will distort the measurements